## Step 0: Load libraries

In [0]:
pip install pyspark

Python interpreter will be restarted.
Collecting pyspark
 Downloading pyspark-3.1.2.tar.gz (212.4 MB)
Collecting py4j==0.10.9
 Downloading py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Building wheels for collected packages: pyspark
 Building wheel for pyspark (setup.py): started
 Building wheel for pyspark (setup.py): finished with status 'done'
 Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=1ad435609a085d8c280803e72039a5b4f9f988fdc93e921cbe42bf169571e4fe
 Stored in directory: /root/.cache/pip/wheels/df/88/9e/58ef1f74892fef590330ca0830b5b6d995ba29b44f977b3926
Successfully built pyspark
Installing collected packages: py4j, pyspark
Successfully installed py4j-0.10.9 pyspark-3.1.2
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28e2694c-0b2f-46ce-8c03-41c517d8ccd7/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
pip install pandas

Python interpreter will be restarted.
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (1.1.5)
Requirement already satisfied: numpy>=1.15.4 in /databricks/python3/lib/python3.8/site-packages (from pandas) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas) (2.8.1)
Requirement already satisfied: pytz>=2017.2 in /databricks/python3/lib/python3.8/site-packages (from pandas) (2020.5)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas) (1.15.0)
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28e2694c-0b2f-46ce-8c03-41c517d8ccd7/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

## Step 1: Load Data

In [0]:
"""
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
url = "jdbc:sqlserver://projectbibda.database.windows.net;DatabaseName=Project"
table = "dbo.Anagrafica_Store"
user = "projectbibda"
password = "xx10!"

Client = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()

Client.show()
"""

## File Vendite : Sales_DG.csv

In [0]:
from pyspark.sql.types import *

file_location = "dbfs:/FileStore/tables/Sales_DG.csv"
delimiter = "\t"

schema =   StructType([StructField('customer', StringType(), True),
            StructField('date', StringType(), True),
            StructField('ticket', StringType(), True),
            StructField('model', StringType(), True),
            StructField('qty', FloatType(), True),
            StructField('sales', StringType(), True)])



# read the training file into a dataframe
sales_csv = spark.read.csv(
  'dbfs:/FileStore/tables/DG/Sales_DG.csv', 
  header=True, 
  schema=schema,
  sep=delimiter
  )

# make the dataframe queriable as a temporary view
sales_csv.createOrReplaceTempView('sales_csv')

# show data
sales_csv.show(5)

+---------+--------+--------------------+----------------+---+--------+
 customer| date| ticket| model|qty| sales|
+---------+--------+--------------------+----------------+---+--------+
DG1000001|01/06/17|1 05020201A ...|F7ZR7TFS54BHWA25|1.0| 311,5|
DG1000001|03/11/17|1 02000201A ...|F66O9TFJRCZN0000|1.0|1.444,44|
DG1000036|08/09/18|1 05020301A ...|F8H93TFH71PHAT06|1.0| 275|
DG1000036|08/09/18|1 05020301A ...|F8K67TFH7VXHWQ99|1.0| 295|
DG1000036|10/05/19|1 05092201A ...|CS1587AK238HWF57|1.0| 745|
+---------+--------+--------------------+----------------+---+--------+
only showing top 5 rows

In [0]:
sales_csv.printSchema()

root
-- customer: string (nullable = true)
-- date: string (nullable = true)
-- ticket: string (nullable = true)
-- model: string (nullable = true)
-- qty: float (nullable = true)
-- sales: string (nullable = true)

In [0]:
from pyspark.sql.functions import *

#substring - rebuild correct date format
sales_view = sales_csv.withColumn('day', concat(lit("20"), substring('date',7,2), lit("-"), substring('date',4,2), lit("-"), substring('date',1,2)))

#modified field "date" into day (datetype) 
sales_view = sales_view.withColumn('day', to_date(sales_view.day, 'yyyy-MM-dd'))


#specify column with comma as decimal and eliminate point for "thousands"

sales_view = sales_view.withColumn('sales', regexp_replace('sales', '\\.', ''))
sales_view = sales_view.withColumn('sales', regexp_replace('sales', ',', '.'))
sales_view = sales_view.withColumn('sales', sales_view['sales'].cast("float"))

#ticket substring 6 char after the first one


sales_view = sales_view.withColumn('ticket', regexp_replace('ticket', ' ', ''))
sales_view = sales_view.withColumn('substr_store', substring('ticket',0,2))

#case when: store from ticket
sales_view = sales_view.withColumn('store', when(sales_view.substr_store == "10", substring('ticket',2,6))
                                 .when(sales_view.substr_store == "1V", substring('ticket',3,6))
                                 .when(sales_view.substr_store == "1O", substring('ticket',3,6))
                                 .when(sales_view.substr_store == "1R", substring('ticket',3,6))
                                 .otherwise("000000"))

# drop columns
columns_to_drop = ['ticket', 'substr_store', 'date']
sales_view = sales_view.drop(*columns_to_drop)


sales_view.createOrReplaceTempView('sales_view')

# show data
sales_view.show(5)


+---------+----------------+---+-------+----------+------+
 customer| model|qty| sales| day| store|
+---------+----------------+---+-------+----------+------+
DG1000001|F7ZR7TFS54BHWA25|1.0| 311.5|2017-06-01|050202|
DG1000001|F66O9TFJRCZN0000|1.0|1444.44|2017-11-03|020002|
DG1000036|F8H93TFH71PHAT06|1.0| 275.0|2018-09-08|050203|
DG1000036|F8K67TFH7VXHWQ99|1.0| 295.0|2018-09-08|050203|
DG1000036|CS1587AK238HWF57|1.0| 745.0|2019-05-10|050922|
+---------+----------------+---+-------+----------+------+
only showing top 5 rows

In [0]:
sales_view.printSchema()

In [0]:
#Check sales by store

"""

%sql
 
SELECT distinct store, sum(sales)
FROM sales_view
Group by store

"""

## File Clienti : Anagrafica_Customer.csv

In [0]:
file_location = "/FileStore/tables/DG/Anagrafica_Customer.csv"
delimiter = ","

schema =   StructType([StructField('customer_crm', StringType(), True),
            StructField('cust_nationality', StringType(), True),
            StructField('cust_country', StringType(), True),
            StructField('cust_birth_date', StringType(), True),
            StructField('cust_gender', StringType(), True),
            StructField('cust_generation', StringType(), True)])



# read the training file into a dataframe
customer = spark.read.csv(
  'dbfs:/FileStore/tables/DG/Anagrafica_Customer.csv', 
  header=True, 
  schema=schema,
  sep=delimiter
  )

# make the dataframe queriable as a temporary view
customer.createOrReplaceTempView('customer')

# show data
customer.show(5)

+------------+----------------+------------------+---------------+-----------+---------------+
customer_crm|cust_nationality| cust_country|cust_birth_date|cust_gender|cust_generation|
+------------+----------------+------------------+---------------+-----------+---------------+
 DG1000001| Europe| DENMARK| Undefined| F| Undefined|
 DG1000036| Europe|RUSSIAN FEDERATION| Undefined| F| Undefined|
 DG1000037| Latin America| MONACO| Undefined| F| Undefined|
 DG1000079| No value| BAHRAIN| 27/06/75| F| Gen X|
 DG1000108| Europe| UKRAINE| 13/05/78| F| Gen X|
+------------+----------------+------------------+---------------+-----------+---------------+
only showing top 5 rows

## File Negozi : Anagrafica_Store.csv

In [0]:
file_location = "/FileStore/tables/DG/Anagrafica_Store.csv"
delimiter = ","

schema =   StructType([StructField('store_cod', StringType(), True),
            StructField('store_desc', StringType(), True),
            StructField('store_region_cod', StringType(), True),
            StructField('store_region_desc', StringType(), True),
            StructField('store_subregion_cod', StringType(), True),
            StructField('store_subregion_desc', StringType(), True),
            StructField('store_country_cod', StringType(), True),
            StructField('store_country_desc', StringType(), True),
            StructField('store_city', StringType(), True),
            StructField('store_status', StringType(), True),
            StructField('store_tipo', StringType(), True),
            StructField('store_target', StringType(), True)])



# read the training file into a dataframe
store = spark.read.csv(
  'dbfs:/FileStore/tables/DG/Anagrafica_Store.csv', 
  header=True, 
  schema=schema,
  sep=delimiter
  )


# make the dataframe queriable as a temporary view
store.createOrReplaceTempView('store')

# show data
store.show(5)


+---------+-------------------+----------------+-----------------+-------------------+--------------------+-----------------+------------------+-----------+------------+----------+------------+
store_cod| store_desc|store_region_cod|store_region_desc|store_subregion_cod|store_subregion_desc|store_country_cod|store_country_desc| store_city|store_status|store_tipo|store_target|
+---------+-------------------+----------------+-----------------+-------------------+--------------------+-----------------+------------------+-----------+------------+----------+------------+
 010001| MILANO SPIGA DONNA| EU| Europe| SE| Southern Europe| IT| ITALY| MILANO| Closed| Retail| Women|
 010002|MILANO VENEZIA UOMO| EU| Europe| SE| Southern Europe| IT| ITALY| MILANO| Open| Retail| Men|
 010003| PORTO CERVO| EU| Europe| SE| Southern Europe| IT| ITALY|PORTO CERVO| Open| Retail| null|
 010004| FIRENZE| EU| Europe| SE| Southern Europe| IT| ITALY| FIRENZE| Open| Retail| null|
 010006| VERONA| EU| Europe| SE| Southern Europe| IT| ITALY| VERONA| Open| Retail| null|
+---------+-------------------+----------------+-----------------+-------------------+--------------------+-----------------+------------------+-----------+------------+----------+------------+
only showing top 5 rows

## File Prodotti : Anagrafica_Prodotto.csv

In [0]:
file_location = "/FileStore/tables/DG/Anagrafica_Prodotto.csv"
delimiter = ";"

schema =   StructType([StructField('prod_brand_desc', StringType(), True),
            StructField('prod_brand_code', StringType(), True),
            StructField('prod_retail_line_group', StringType(), True),
            StructField('prod_line_desc', StringType(), True),
            StructField('prod_line_code', StringType(), True),
            StructField('prod_commercial_class_desc', StringType(), True),
            StructField('prod_commercial_class_code', StringType(), True),
            StructField('prod_model_desc', StringType(), True),
            StructField('prod_model_desc_2', StringType(), True),
            StructField('prod_model_desc_3', StringType(), True),
            StructField('prod_model_code', StringType(), True),
            StructField('prod_color_desc', StringType(), True),
            StructField('prod_color_code', StringType(), True),
            StructField('prod_model_part_color_code', StringType(), True),
            StructField('prod_article_desc', StringType(), True)])



# read the training file into a dataframe
product = spark.read.csv(
  'dbfs:/FileStore/tables/DG/Anagrafica_Prodotto.csv', 
  header=True, 
  schema=schema,
  sep=delimiter
  )

product = product.withColumn('prod_article_desc', regexp_replace('prod_article_desc', '"', '_'))

product = product.withColumn('prod_commercial_class_desc', when(product.prod_commercial_class_desc == "FORMALE", "SHOES FORMALE")
                                                           .otherwise(product.prod_commercial_class_desc))

# make the dataframe queriable as a temporary view
product.createOrReplaceTempView('product')

# show data
product.show(5)

+---------------+---------------+----------------------+--------------+--------------+--------------------------+--------------------------+---------------+-----------------+-----------------+---------------+-----------------+---------------+--------------------------+-----------------+
prod_brand_desc|prod_brand_code|prod_retail_line_group|prod_line_desc|prod_line_code|prod_commercial_class_desc|prod_commercial_class_code|prod_model_desc|prod_model_desc_2|prod_model_desc_3|prod_model_code| prod_color_desc|prod_color_code|prod_model_part_color_code|prod_article_desc|
+---------------+---------------+----------------------+--------------+--------------+--------------------------+--------------------------+---------------+-----------------+-----------------+---------------+-----------------+---------------+--------------------------+-----------------+
 AM-AS-AG| 11.0| WOMAN| APPAREL WOMAN| 2.0| ALTA MODA| AM| ABITO| ABITO| ABITO WOMAN| F6190S|VARIANTE ABBINATA| S9000| F6190SG3058S9000| ABITO|
 AM-AS-AG| 11.0| WOMAN| APPAREL WOMAN| 2.0| ALTA MODA| AM| ABITO| ABITO| ABITO WOMAN| F6208S|VARIANTE ABBINATA| S9000| F6208SG3103S9000| ABITO|
 AM-AS-AG| 11.0| WOMAN| APPAREL WOMAN| 2.0| ALTA MODA| AM| ABITO| ABITO| ABITO WOMAN| F6327S|VARIANTE ABBINATA| S9000| F6327SGV029S9000| ABITO|
 AM-AS-AG| 11.0| WOMAN| APPAREL WOMAN| 2.0| ALTA MODA| AM| ABITO| ABITO| ABITO WOMAN| F6371S| STAMPA| X0800| F6371SFPRCPX0800| ABITO|
 AM-AS-AG| 11.0| WOMAN| APPAREL WOMAN| 2.0| ALTA MODA| AM| ABITO| ABITO| ABITO WOMAN| F6519S|VARIANTE ABBINATA| S9000| F6519SGV249S9000| ABITONE|
+---------------+---------------+----------------------+--------------+--------------+--------------------------+--------------------------+---------------+-----------------+-----------------+---------------+-----------------+---------------+--------------------------+-----------------+
only showing top 5 rows

### costruisco id scontrino : negozio-cliente-giorno

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

SQL_Transaction = "SELECT store||customer||day||model as Ticket_Row, store||customer||day as Ticket, * FROM sales_view left join store on store = store_cod left join customer on customer = customer_crm inner join product on model = prod_model_part_color_code"

sales_dg = sqlContext.sql(SQL_Transaction)

#sales_dg.show(5)

In [0]:
sales_dg.printSchema()

root
-- Ticket_Row: string (nullable = true)
-- Ticket: string (nullable = true)
-- customer: string (nullable = true)
-- model: string (nullable = true)
-- qty: float (nullable = true)
-- sales: float (nullable = true)
-- day: date (nullable = true)
-- store: string (nullable = true)
-- store_cod: string (nullable = true)
-- store_desc: string (nullable = true)
-- store_region_cod: string (nullable = true)
-- store_region_desc: string (nullable = true)
-- store_subregion_cod: string (nullable = true)
-- store_subregion_desc: string (nullable = true)
-- store_country_cod: string (nullable = true)
-- store_country_desc: string (nullable = true)
-- store_city: string (nullable = true)
-- store_status: string (nullable = true)
-- store_tipo: string (nullable = true)
-- store_target: string (nullable = true)
-- customer_crm: string (nullable = true)
-- cust_nationality: string (nullable = true)
-- cust_country: string (nullable = true)
-- cust_birth_date: string (nullable = true)
-- cust_gender: string (nullable = true)
-- cust_generation: string (nullable = true)
-- prod_brand_desc: string (nullable = true)
-- prod_brand_code: string (nullable = true)
-- prod_retail_line_group: string (nullable = true)
-- prod_line_desc: string (nullable = true)
-- prod_line_code: string (nullable = true)
-- prod_commercial_class_desc: string (nullable = true)
-- prod_commercial_class_code: string (nullable = true)
-- prod_model_desc: string (nullable = true)
-- prod_model_code: string (nullable = true)
-- prod_color_desc: string (nullable = true)
-- prod_color_code: string (nullable = true)
-- prod_model_part_color_code: string (nullable = true)
-- prod_article_desc: string (nullable = true)

In [0]:
sales_dg.write.mode("overwrite").option("delimiter", "\t").option("decimal", ",").option("encoding", "ISO-8859–1").saveAsTable("olap_sales_table")

In [0]:
%sql

select distinct prod_commercial_class_desc from olap_sales_table;
select count(*) from olap_sales_table

count(1)
5824001


In [0]:
sales_dg.coalesce(1).write.mode("overwrite").format("com.databricks.spark.csv").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/sales_dg.csv")

In [0]:
#dbutils.fs.rm("dbfs:/FileStore/tables/Anagrafica_Prodotto.csv")

Out[1]: False

## Step 2: Market Basket Analysis

In [0]:
import pandas as pd

In [0]:
from pyspark import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import functions as F

from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType

from pyspark.ml.fpm import FPGrowth

#### unità: scontrino (cliente-giorno-negozio) , item : classe commerciale

In [0]:
#raggruppo
groups_class = sales_dg.groupby(['Ticket']).agg(F.collect_set('prod_commercial_class_desc').alias('items'))
#tolgo i record con solo 1 item
groups_class = groups_class.select('*',size('items').alias('n_items'))
groups_class_no_singoli = groups_class.filter(groups_class.n_items > 1)
#salvo i risultati

groups_class.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/item_by_commercial_class.json")
groups_class_no_singoli.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/item_by_commercial_class_no1.json")

##### unità: scontrino (cliente-giorno-negozio) , item : model desc 2 (prodotto riclassificato)

In [0]:
#raggruppo
groups_model_2 = sales_dg.groupby(['Ticket']).agg(F.collect_set('prod_model_desc_2').alias('items'))
#tolgo i record con solo 1 item
groups_model_2 = groups_model_2.select('*',size('items').alias('n_items'))
groups_model_2_no_singoli = groups_model_2.filter(groups_model_2.n_items > 1)
#salvo i risultati
groups_model_2.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/item_by_model_desc_2.json")
groups_model_2_no_singoli.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/item_by_model_desc_2_no1.json")

##### unità: scontrino (cliente-giorno-negozio) , item : model desc 3 (prodotto riclassificato + MAN/WOM/BABY/)

In [0]:
#raggruppo
groups_model_3 = sales_dg.groupby(['Ticket']).agg(F.collect_set('prod_model_desc_3').alias('items'))
#tolgo i record con solo 1 item
groups_model_3= groups_model_3.select('*',size('items').alias('n_items'))
groups_model_3_no_singoli = groups_model_3.filter(groups_model_3.n_items > 1)
#salvo i risultati
groups_model_3.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/item_by_model_desc_3.json")
groups_model_3_no_singoli.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/item_by_model_desc_3_no1.json")

### MBA 1 : item è la classe commerciale

In [0]:
#modifico solo il valore di groups, così il resto rimane uguale
groups = groups_class
groups_no_singoli = groups_class_no_singoli

In [0]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01)
model = fpGrowth.fit(groups)

# Display frequent itemsets.
fp = model.freqItemsets
#fp.sort(fp.freq.desc()).show(n=20)

#! salvare questi output in csv !
regole=model.associationRules

#### regole con dataset filtrato : item > 1 ####

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01)
model = fpGrowth.fit(groups_no_singoli)

# Display frequent itemsets.
fp = model.freqItemsets
#fp.sort(fp.freq.desc()).show(n=20)

#! salvare questi output in csv !
regole_no_singoli=model.associationRules
#regole.sort(regole.support.desc()).show()


### salvo output regole create MBA 1

In [0]:
regole.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/regole_commercial_class.json")
regole_no_singoli.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/regole_commercial_class_no1.json")

In [0]:
#COMANDI PER MOSTRARE A VIDEO I RISULTATI (disattivati per ora)

#regole.sort(regole.support.desc()).show()
#model.associationRules.show()
#model.transform(groups).show()

### MBA 2 : item il model_desc_2 (prodotto riclassificato)

In [0]:
#modifico solo il valore di groups, così il resto rimane uguale
groups = groups_model_2
groups_no_singoli = groups_model_2_no_singoli

In [0]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01)
model = fpGrowth.fit(groups)

# Display frequent itemsets.
fp = model.freqItemsets
#fp.sort(fp.freq.desc()).show(n=20)

#! salvare questi output in csv !
regole=model.associationRules
#regole.sort(regole.support.desc()).show()

#### regole con dataset filtrato : item > 1 ####

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01)
model = fpGrowth.fit(groups_no_singoli)

# Display frequent itemsets.
fp = model.freqItemsets
#fp.sort(fp.freq.desc()).show(n=20)

#! salvare questi output in csv !
regole_no_singoli=model.associationRules
#regole.sort(regole.support.desc()).show()

### salvo output regole create MBA 2

In [0]:
regole.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/regole_model_desc_2.json")
regole_no_singoli.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/regole_model_desc_2_no1.json")

### MBA 3 : item il model_desc_2 (prodotto riclassificato+WOM/MAN/BABY)

In [0]:
#modifico solo il valore di groups, così il resto rimane uguale
groups = groups_model_3 
groups_no_singoli = groups_model_3_no_singoli

In [0]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01)
model = fpGrowth.fit(groups)

# Display frequent itemsets.
fp = model.freqItemsets
#fp.sort(fp.freq.desc()).show(n=20)

#! salvare questi output in csv !
regole=model.associationRules
#regole.sort(regole.support.desc()).show()

#### regole con dataset filtrato: item > 1 ####

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01)
model = fpGrowth.fit(groups_no_singoli)

# Display frequent itemsets.
fp = model.freqItemsets
#fp.sort(fp.freq.desc()).show(n=20)

#! salvare questi output in csv !
regole_no_singoli=model.associationRules
#regole.sort(regole.support.desc()).show()

### salvo output regole create MBA 3

In [0]:
regole.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/regole_model_desc_3.json")
regole_no_singoli.coalesce(1).write.mode("overwrite").format("json").option("delimiter", "\t").option("header", "true").save("dbfs:/FileStore/df/regole_model_desc_3_no1.json")